<a href="https://colab.research.google.com/github/mialli963/PHYS3025Work/blob/main/PHYS_3025_Activity_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is adapted from Python tutorials by P. L. Lim, Adrian Price-Whelan, Kelle Cruz, and Stephanie T. Douglas with the AstroPy Foundation. I've modified them a bit to match our needs.
For the original cone search tutorial you can visit: https://learn.astropy.org/tutorials/conesearch.html . I've updated it to avoid errors and didn't use the last part but you can look there to see how to search more than one catalog at once and time the searches as well.
For the rest of the tutorial I based it on https://learn.astropy.org/tutorials/plot-catalog.html but the text files aren't available so we're using search methods to get data.

In [ ]:
# Python standard library
import time
import warnings

# Third-party software
import numpy as np

# Astropy
from astropy import coordinates as coord
from astropy import units as u
from astropy.table import Table

# Astroquery. This tutorial requires 0.3.5 or greater.
!pip install astroquery #note we need to install it manually using !pip install
import astroquery
from astroquery.simbad import Simbad
from astroquery.vo_conesearch import conf, conesearch, vos_catalog

# Set up matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.9/888.9 kB 42.2 MB/s eta 0:00:00


We'll be using ConeSearch first. To start, it might be useful to list the available Cone Search catalogs first. By default, catalogs that pass nightly validation are included. Validation is hosted by Space Telescope Science Institute (STScI).

In [ ]:
conesearch.list_catalogs()

['2MASS All-Sky Catalog of Point Sources 1',
 'Gaia DR2 5',
 'Guide Star Catalog 2.3 Cone Search 1',
 'SDSS DR7 - Sloan Digital Sky Survey Data Release 7 1',
 'SDSS DR7 - Sloan Digital Sky Survey Data Release 7 2',
 'SDSS DR7 - Sloan Digital Sky Survey Data Release 7 3',
 'SDSS DR7 - Sloan Digital Sky Survey Data Release 7 4',
 'The HST Guide Star Catalog, Version 1.1 1',
 'The HST Guide Star Catalog, Version 1.2 1',
 'The HST Guide Star Catalog, Version GSC-ACT 1',
 'The PMM USNO-A1.0 Catalogue 1',
 'The USNO-A2.0 Catalogue 1',
 'The USNO-B1.0 Catalog 2',
 'Two Micron All Sky Survey (2MASS) 1',
 'Two Micron All Sky Survey (2MASS) 2']

Next, pick an astronomical object of interest. For example, M31.

In [ ]:
c = coord.SkyCoord.from_name('Wasp-43b', frame='icrs')
print(c)

<SkyCoord (ICRS): (ra, dec) in deg
    (154.9083706, -9.80627864)>




By default, a basic Cone Search goes through the list of catalogs and stops at the first one that returns non-empty VO table. Let's search for objects within 0.1 degree around M31. You will see a lot of warnings that were generated by VO table parser but ignored by Cone Search service validator. VO compliance enforced by Cone Search providers is beyond the control of astroquery.vo_conesearch package.

The result is an[ Astropy](https://http://astropy.readthedocs.io/en/stable/table/index.html) table.


In [ ]:
result = conesearch.conesearch(c, 0.1 * u.degree)

Trying http://vizier.unistra.fr/viz-bin/conesearch/II/246/out?


In [ ]:
print('First non-empty table returned by', result.url)
print('Number of rows is', len(result))

First non-empty table returned by http://vizier.unistra.fr/viz-bin/conesearch/II/246/out?
Number of rows is 75


In [ ]:
print(result)

   _r     RAJ2000    DEJ2000   errMaj errMin ... scanKey coaddKey coadd Opt
            deg        deg     arcsec arcsec ...                           
-------- ---------- ---------- ------ ------ ... ------- -------- ----- ---
0.082097 154.904556  -9.888290   0.06   0.06 ...   14322   329391    92 Opt
0.093857 154.934859  -9.896434   0.06   0.06 ...   14322   329391    92 Opt
0.082946 154.913709  -9.889058   0.07   0.07 ...   14322   329391    92 Opt
0.097131 154.969373  -9.882580   0.06   0.06 ...   14323   329421   174 Opt
0.064090 154.871025  -9.858752   0.06   0.06 ...   14322   329391    92 Opt
0.077042 154.849258  -9.856708   0.31   0.28 ...   14322   329391    92 Opt
0.043236 154.868706  -9.824766   0.14   0.14 ...   14322   329392   103 Opt
0.060900 154.856162  -9.838873   0.20   0.17 ...   14322   329392   103 Opt
0.058465 154.851253  -9.822106   0.08   0.08 ...   14322   329392   103 Opt
0.099336 154.816377  -9.846916   0.07   0.06 ...   14321   329375   174 Opt
     ...    

This table can be manipulated like any other Astropy table; e.g., re-write the table into LaTeX format.

In [ ]:
result.write('my_result.tex', format='ascii.latex', overwrite=True)



You can now use your favorite text editor to open the my_result.tex file, but here, we are going to read it back into another Astropy table.

Note that the extra data_start=4 option is necessary due to the non-roundtripping nature of LaTeX reader/writer (see astropy issue [5205](https://github.com/astropy/astropy/issues/5205)).


In [ ]:
result_tex = Table.read('my_result.tex', format='ascii.latex', data_start=4)
print(result_tex)


Cone Search results can also be used in conjuction with other types of queries. For example, you can query SIMBAD for the first entry in your result above.


In [ ]:
# Due to the unpredictability of external services,
# The first successful query result (above) might differ
# from run to run.
#
# CHANGE THESE VALUES to the appropriate RA and DEC
# column names you see above, if necessary.
# These are for http://gsss.stsci.edu/webservices/vo/ConeSearch.aspx?CAT=GSC23&
ra_colname = 'RAJ2000'
dec_colname = 'DEJ2000'
# Don't run this cell if column names above are invalid.
if ra_colname in result.colnames and dec_colname in result.colnames:
    row = result[0]
    simbad_obj = coord.SkyCoord(ra=row[ra_colname]*u.deg, dec=row[dec_colname]*u.deg)
    print('Searching SIMBAD for\n{}\n'.format(simbad_obj))
    simbad_result = Simbad.query_region(simbad_obj, radius=5*u.arcsec)
    print(simbad_result)
else:
    print('{} or {} not in search results. Choose from: {}'.format(
        ra_colname, dec_colname, ' '.join(result.colnames)))

Now back to Cone Search... You can extract metadata of this Cone Search catalog.

In [ ]:
my_db = vos_catalog.get_remote_catalog_db(conf.conesearch_dbname)
my_cat = my_db.get_catalog_by_url(result.url)
print(my_cat.dumps())

If you have a favorite catalog in mind, you can also perform Cone Search only on that catalog. A list of available catalogs can be obtained by calling conesearch.list_catalogs(), as mentioned above. For example 'The USNO-A2.0 Catalogue 1'

In [ ]:
try:
    result = conesearch.conesearch(
        c, 0.1 * u.degree, catalog_db='The USNO-A2.0 Catalogue 1')
except Exception as e:
    # We provide a cached version of the result table in case the query fails
    # due to an intermittent server-side issue, or if you do not have an
    # internet connection
    result = Table.read('usno-A2-result.fits')

print('Number of rows is', len(result))



Let's explore the 3 rows of astronomical objects found within 0.1 degree of M31 in the given catalog and sort them by increasing distance. For this example, the VO table has several columns that might include:

    _r = Angular distance (in degrees) between object and M31
    USNO-A2.0 = Catalog ID of the object
    RAJ2000 = Right ascension of the object (epoch=J2000)
    DEJ2000 = Declination of the object (epoch=J2000)

Note that column names, meanings, order, etc. might vary from catalog to catalog.


In [ ]:
col_names = result.colnames
print(col_names)



In [ ]:
# Before sort
print(result)



In [ ]:
# After sort
result.sort('_r')
print(result)



Now let's do some astrophysics. Suppose we wanted to make a plot

Let's look at stars in the  Perseus OB-1 double cluster. We're going to do a cone search of stars in the Gaia catalog ('Gaia DR2 5') and save to an ascii. Note that the search needs a name in the proper format so we're looking for NGC 869/884 which is one catalog name of the double cluster. It's pretty large but let's look within a tenth of a degree of the the cluster. To get the commands, use the examples we just did as a starting point. Also, print the column names as it will help you later.

Let's save this as a csv.

In [ ]:
result.write('my_result.csv', format='ascii.csv', overwrite=True)

Now let's read it. Note that if had a downloaded csv it might be a different command. See https://learn.astropy.org/tutorials/plot-catalog.html.

In [ ]:
data = Table.read("my_result.csv")

Let's see the parallax column.

In [ ]:
data['RPlx']

Note that we need to get rid of the negative parallaxes which are not physical. In reality we should figure out why they're negative but for now we'll just skip them. Use the command below.

In [ ]:
data_clean=data[data["Plx"]>=0]

Covert the parallax column to distance in parsecs and as an answer give the first three and last three distances in the table (that is distance of stars 0-2 and -3->-1). Note that the unit of parallax for Gaia is mas (milli-arcsecond).

Oh no! Not all the distances are the same as we'd expect for a star cluster (we'll talk about that!). I wonder if there's a difference between stars in the cluster and stars outside of it. Let's plot Right Ascension (RAJ2000) vs Declination (DEJ2000) and use distance as a color axis to see if we ca find where those stars are. I have the paritial code below but it needs you to have calculated distance. Note we have to use log10 because of the spread between distances.

In [ ]:
plt.scatter(data_clean["RAJ2000"],data_clean["DEJ2000"],c=np.log10(distance)) # plot J-K vs. J
cbar=plt.colorbar()
cbar.set_label('Log10(Distance)')
plt.ylim(reversed(plt.ylim())) # flip the y-axis
plt.xlabel("RA (deg)", fontsize=20)
plt.ylabel("DEC (deg)", fontsize=20)



Okay, so that didn't really help! Try plotting either proper motion (pmRA vs pmDE) or Radial Velocity (RV) vs distance and write about what you observe. It's a bit tricky to filter out, isn't it!

After that, make a color-magnitude diagram by plotting BP-RP on X and absolute Gmag on Y. Let's keep distance as the color dimension.  Keep in mind you'll need to calculate the absolute Gmag using the equaions from class! We'll talk more about what this plot says later in class.

After this we'll do a few more plots. Let's plot Temperature (Teff) vs Luminisoity (Lum); Radius (Rad) vs Luminosity, and Color (BP-RP) vs Teff. Write a few sentences about what you observe.